# Проект. Исследование надежности заемщиков
## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

------

## Этап 1. Предобработка данных
### Общая информация о данных

In [1]:
import pandas as pd

Прочитаем файл data.csv и сохраним его в переменной data.

In [2]:
data = pd.read_csv('/datasets/data.csv')

Получение первых 10 строк таблицы.

In [3]:
data.head(10)

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы data.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Значение | Описание
---------|---------
children | количество детей в семье;
days_employed | трудовой стаж в днях;
dob_days | возраст клиента в годах;
education | образование клиента;
education_id | идентификатор образования;
family_status | семейное положение;
family_status_id | идентификатор семейного положения;
gender | пол клиента;
income_type | тип занятости;
debt | имел ли задолженность по возврату кредитов;
total_income | доход в месяц;
purpose | цель получения кредита;

Всего в таблице имеется 12 столбцов. Количество значений в столбцах 'days_employed' и 'total_income' отличаются, значит есть пропущенные значения. Также в стоблце 'days_employed' присутствуют отрицательные значения. В столбце 'education' различается регистр значений.

### Работа с пропусками

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
data.isnull().sum()

children               0
days_employed       2174
dob_days               0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружили пропуски в количественных переменных 'days_employed' и 'total_income'.

Проверим, случайны ли данные пропуски.

In [6]:
print(data[(data['days_employed'].isnull()) & (data['total_income'].isnull())])

       children  days_employed  dob_days            education  education_id  \
12            0            NaN        65              среднее             1   
26            0            NaN        41              среднее             1   
29            0            NaN        63              среднее             1   
41            0            NaN        50              среднее             1   
55            0            NaN        54              среднее             1   
65            0            NaN        21              среднее             1   
67            0            NaN        52               высшее             0   
72            1            NaN        32               высшее             0   
82            2            NaN        50               высшее             0   
83            0            NaN        52              среднее             1   
90            2            NaN        35               высшее             0   
94            1            NaN        34            

У нас получилась выборка, в которой 2174 строки, следовательно пропуски не случайны: клиенты, которые не указали трудовой стаж, также не указали уровень дохода в месяц.

Займемся заполнением пропусков, но для начала необходимо сделать отрицательные значения столбца 'days_employed' положительными.

Применим метод abs() к столбцу 'days_employed'.

In [8]:
data['days_employed'] = data['days_employed'].abs()

Удостоверимся, что в столбце 'days_employed' теперь только положительные значения.

In [9]:
data[data['days_employed'] < 0]['days_employed'].count() # с помощью метода .count() посчитаем количество отрицательных значений 

0

Так как в задании проекта нужно определить влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок, то без особого вреда на результат можно заменить пропущенные значения столбцов 'days_employed' и 'total_income' средним арифметическим значений или медианой.

Найдем среднее арифметическое значение для значений столбца 'days_employed'.

In [10]:
days_employed_mean = data['days_employed'].mean()
print(days_employed_mean)

66914.72890682236


Найдем медиану для этих же значений.

In [11]:
days_employed_median = data['days_employed'].median()
print(days_employed_median)

2194.220566878695


Значение медианы и среднего арифметического различаются в разы. Это означает, что в выборке есть очень большие значения, которые делают среднее арифметическое не соответствующим реальности. Для замены пропущенных значений будем использовать медиану. Для замены пропусков будем использовать метод .fillna с аргументом (value='').

In [12]:
data['days_employed'] = data['days_employed'].fillna(value=days_employed_median)

Теперь проделаем те же операции для столбца 'total_income'.
Найдем среднее арифметическое значение для столбца  'total_income'. 

In [13]:
total_income_mean = data['total_income'].mean()
print(total_income_mean)

167422.30220817294


Найдем медиану для этих же значений.

In [14]:
total_income_median = data['total_income'].median()
print(total_income_median)

145017.93753253992


В данном случае значения среднего арифметического и медианы одного порядка. Для замены пропусков возьмем значение среднего арифметического.

In [15]:
data['total_income'] = data['total_income'].fillna(value=total_income_mean)

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [16]:
data.isnull().sum()

children            0
days_employed       0
dob_days            0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Мы избавились от пропусков в данных.

### Изменение типов данных

Изменим вещественный тип данных на целочисленный в столбцах 'days_employed' и 'total_income'. Применим метод astype('int').

In [17]:
data['days_employed'] = data['days_employed'].astype('int')

In [18]:
data['total_income'] = data['total_income'].astype('int')

Посмотрим, что у нас получилось.

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [20]:
data.head(10)

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Проверка данных на адекватность и обработка дубликатов

Исследуем на адекватность важные для нас данные, начнем со столбцов 'family_status' и 'family_status_id'. Применим метод value_counts().

**Оставляю переменную data_mod, так как после этого момента я проводил все операции с ней.**

In [25]:
data_mod = data

In [26]:
data_mod['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [27]:
data_mod['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Данные пригодны для анализа.

Выведем информацию о таблице data_mod.

In [28]:
data_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Теперь исследуем стобец 'children'. Применим к столбцу метод value_counts().

In [29]:
data_mod['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Среди значений столбца 'children' есть неадекватные данные, нужно принять это во внимание при категоризации значений.

Теперь займемся удалением дубликатов из таблицы. Но для начала сделаем значения в столбце 'education' в нижнем регистре с помощью метода .str.lower().

In [30]:
data_mod.education = data.education.str.lower() # меняем регистр значений столбца, используя исходную таблицу

Проверим, что у нас получилось.

In [31]:
data_mod['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь регистр значений столбца 'education' в порядке.

Также перед удалением дубликатов необходимо посмотреть текстовые значения столбцов 'family_status', 'income_type', 'purpose'.

In [32]:
data_mod['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [33]:
data_mod['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [34]:
data_mod['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Данные в порядке, можно приступать к удалению дубликатов.

Найдем дубликаты в таблице комбинацией методов duplicated() и sum().

In [35]:
data_mod.duplicated().sum()

71

Попробуем разобраться, откуда могли взяться дубликаты.

Создадим в таблице data_mod столбец 'duplicates', в котором будет значение True, если это дубликат и False, если нет.

In [36]:
data_mod['duplicates'] = data_mod.duplicated()

Теперь создадим новую таблицу data_duplicate, которая будет состоять из строк, которые являются дубликатами.

In [37]:
data_duplicate = data_mod[data_mod['duplicates'] != False]

Выведем эту таблицу на экран.

In [38]:
data_duplicate

,children,days_employed,dob_days,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,duplicates
2849,0,2194,41,среднее,1,женат / замужем,0,F,сотрудник,0,167422,покупка жилья для семьи,True
3290,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу,True
4182,1,2194,34,высшее,0,гражданский брак,1,F,сотрудник,0,167422,свадьба,True
4851,0,2194,60,среднее,1,гражданский брак,1,F,пенсионер,0,167422,свадьба,True
5557,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,167422,сыграть свадьбу,True
6312,0,2194,30,среднее,1,женат / замужем,0,M,сотрудник,0,167422,строительство жилой недвижимости,True
7808,0,2194,57,среднее,1,гражданский брак,1,F,пенсионер,0,167422,на проведение свадьбы,True
7921,0,2194,64,высшее,0,гражданский брак,1,F,пенсионер,0,167422,на проведение свадьбы,True
7938,0,2194,71,среднее,1,гражданский брак,1,F,пенсионер,0,167422,на проведение свадьбы,True
8583,0,2194,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,167422,дополнительное образование,True


Можно заметить что значения столбца *'days_employed'* одинаковы, и значения столбца *'total_income'* одинаковы.

Проверим это, применив к столбцам метод `value_counts()`.

In [39]:
data_duplicate['days_employed'].value_counts()

2194    71
Name: days_employed, dtype: int64

In [40]:
data_duplicate['total_income'].value_counts()

167422    71
Name: total_income, dtype: int64

Итак, эти значения действительно одинаковые и равны значениям медианы и среднего арифметического, которые были посчитаны для замены пропусков в данных. Теперь понятно, откуда взялись эти строки - дубликаты. В общем, дубликаты выглядят странно, если учесть, что у каждого клиента свой трудовой стаж и свой доход в месяц, и их можно было бы смело удалять, так как это была бы скорей всего ошибка в данных. Но в данном случае, на месте трудового стажа и дохода в месяц были пропуски, а они были заменены на одинаковые значения. Вероятность возникновения дубликатов возросла, и они появились. Поэтому дубликаты в данной ситуации удалять не нужно.

In [43]:
data_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
duplicates          21525 non-null bool
dtypes: bool(1), int64(7), object(5)
memory usage: 2.0+ MB


### Лемматизация

Данные готовы для анализа. Выделим леммы для столбца 'purpose'. С помощью метода unique() получим список уникальных значений столбца 'purpose'.

In [44]:
lemmas_purpose = data_mod['purpose'].unique()

Выведем на экран данный список уникальных значений.

In [45]:
lemmas_purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Лемматизируем список lemmas_purpose. Используем библиотеку pymystem3.

In [46]:
from pymystem3 import Mystem        # импортируем библиотеку pymystem3
m = Mystem()

In [47]:
lemmas_words = []                   # создаем пустой список, который будем использовать для хранения лемм

In [48]:
for element in lemmas_purpose:      # с помощью цикла лемматизируем каждый элемент списка lemmas_purpose
    lemmas = m.lemmatize(element)   
    lemmas_words += lemmas          # "складываем" лемматизированные значения в один список
    print(lemmas)                   # выводим результат на экран

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

Мы получили леммы уникальных значений столбца *'purpose'*. Теперь посчитаем количество уникальных значений в списке *lemmas_words*. Для этого используем контейнер *Counter*.

In [49]:
from collections import Counter # вызываем контейнер Counter из модуля collections
Counter(lemmas_words)           

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

Выбираем для словаря *lemma_dict* значимые и чаще всех встречающиеся леммы из списка - счетчика. Это леммы *жилье*, *автомобиль*, *образование*, *недвижимость* и *свадьба*. Леммы *покупка*, *операция*, *строительство*, *свой* встречаются чаще некоторых выбранных значений для словаря, но по своей сути не являются значимыми, то есть не несут информации о самой цели кредита.

In [50]:
lemma_dict = ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость']

Отсортируем список методом sort().

In [51]:
lemma_dict.sort()

Проверим результат, выведем список на экран.

In [52]:
lemma_dict

['автомобиль', 'жилье', 'недвижимость', 'образование', 'свадьба']

Сейчас необходимо перебором сравнить каждую лемматизированную строку столбца 'purpose' со значениями словаря lemma_dict и добавить найденное значение в новый столбец 'purpose_lemm'.

Напишем функцию purpose_lemm, которая принимает строку-аргумент, лемматизирует ее, затем ищет соответствие каждой леммы значению словаря lemma_dict и при нахождении соответствия, возващает эту лемму.

In [53]:
def purpose_lemm(row):
    words = m.lemmatize(row)
    for element in words:
        if element in lemma_dict:
            return element

Применим функцию purpose_lemm для каждой строки столбца 'purpose'.

In [54]:
data_mod['purpose_lemm'] = data_mod['purpose'].apply(purpose_lemm)

Посмотрим, что у нас получилось. Используем метод value_counts().

In [55]:
data_mod['purpose_lemm'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_lemm, dtype: int64

Мы получили первые результаты нашего исследования. Выводы будем делать позже.

### Категоризация данных

Теперь перейдем к категоризации, для того чтобы ответить на вопросы проекта. Начнем со столбца 'children'. Применим к столбцу метод value_counts().

In [56]:
data_mod['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Итак, среди значений столбца есть "неправильные" данные. Это значения "-1" и "20", возможно, в первом случае по ошибке добавлен знак минус, а во втором - ноль, но мы не будем учитывать эти значения. Напишем функцию children_categories для категоризации значений, на выходе получим две категорий значений: 'есть дети' и 'нет детей'.

In [57]:
def children_categories(quantity):
    if quantity == 0:
        return 'нет детей'
    if 0 < quantity <= 5:
        return 'есть дети'
    

Применим функцию children_categories к столбцу 'children' и создадим новый столбец 'children_categories' в таблице data_mod.

In [58]:
data_mod['children_categories'] = data_mod['children'].apply(children_categories)

Применим теперь метод value_counts() к столбцу 'children_categories', посмотрим что у нас получилось.

In [59]:
data_mod['children_categories'].value_counts()

нет детей    14149
есть дети     7253
Name: children_categories, dtype: int64

Построим новую таблицу children_debt. Группируем значения 'children_categories' с помощью метода groupby(), далее с помощью метода agg() добавляем еще два столбца: в первом количество значений данной категории, во втором сумма невыплаченных кредитов в данной категории.

In [60]:
children_debt = data_mod.groupby('children_categories').agg({'children':['count'],'debt': ['sum']})

Добавим в таблицу children_debt новый столбец 'ratio', как отношение числа невыплаченных кредитов к числу значений в данной категории.

In [61]:
children_debt['ratio'] = children_debt['debt']['sum'] / children_debt['children']['count']

Выведем таблицу children_debt на экран.

In [62]:
children_debt

,children,debt,ratio
,count,sum,
children_categories,,,
есть дети,7253,669,0.092238
нет детей,14149,1063,0.075129


Анализ таблицы будет дан в заключении проекта.

Переходим к категоризации значений столбца 'family_status'. Для начала посмотрим уникальные значения столбца, используем метод value_counts().

In [63]:
data_mod['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Данные нас устраивают, можно переходить сразу к построению таблицы family_status_debt. Группируем значения столбца 'family_status' с помощью метода groupby(), далее с помощью метода agg() добавляем еще два столбца: в первом количество значений данной категории, во втором сумма невыплаченных кредитов в данной категории.

In [64]:
family_status_debt = data_mod.groupby('family_status').agg({'family_status':['count'],'debt': ['sum']})

Добавим в таблицу family_status_debt новый столбец 'ratio', как отношение числа невыплаченных кредитов к числу значений в данной категории.

In [65]:
family_status_debt['ratio'] = family_status_debt['debt']['sum'] / family_status_debt['family_status']['count']

Выведем таблицу family_status_debt на экран, отсортируя ее методом sort_values с аргументом 'ratio'. Также с помощью аргумента ascending=False сделаем сортировку по убыванию.

In [66]:
family_status_debt.sort_values(by='ratio', ascending=False)

,family_status,debt,ratio
,count,sum,
family_status,,,
Не женат / не замужем,2813,274,0.097405
гражданский брак,4177,388,0.092890
женат / замужем,12380,931,0.075202
в разводе,1195,85,0.071130
вдовец / вдова,960,63,0.065625


Отлично, мы получили таблицу, показывающую зависимость семейного положения на возврат кредита.

Теперь необходимо категоризировать значения столбца 'total_income'. Проблема в том, что у нас нет точных значений уровня дохода для того, чтобы сделать сравнение по категорям.

Чтобы найти адекватные значения уровня дохода для сравнения по категориям, посмотрим минимальное, максимальное значения и медиану.

In [67]:
data_mod['total_income'].min()

20667

In [68]:
data_mod['total_income'].max()

2265604

In [69]:
data_mod['total_income'].median()

156400.0

Для определения значений, которые будут разделять категории значений не хватает информации. Поэтому для определения категорий клиентов по уровню дохода возьмем другой столбец 'income_type'. Для начала применим к столбцу метод value_counts().

In [70]:
data_mod['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Создадим новую таблицу *data_income_categories*, в которой будут выводиться средние значения дохода клиентов в месяц в зависимости от типа занятости клиентов. Исользуем метод `groupby()` для группировки и `mean()` для нахождения среднего арифметического.

In [71]:
data_income_categories = data_mod.groupby('income_type')['total_income'].mean()

Отсортируем таблицу методом `sort_values()` и выведем ее на экран.

In [72]:
data_income_categories.sort_values(ascending=False)

income_type
предприниматель    333292.500000
компаньон          198920.912488
госслужащий        170547.597670
сотрудник          161980.230956
пенсионер          140371.744554
безработный        131339.000000
студент             98201.000000
в декрете           53829.000000
Name: total_income, dtype: float64

Проанализировав полученные данные, принимаем средний уровень дохода, если клиент *сотрудник*, значение - 'низкий', если клиент - *студент, в декрете, безработный или пенсионер*, и значение 'высокий' в остальных случаях.

Напишем функцию income_type, которая будет выполнять требуемую категоризацию.

In [73]:
def income_type(types):
    if types == 'сотрудник':
        return 'средний'
    if types == 'компаньон' or types == 'госслужащий' or types == 'предприниматель':
        return 'высокий'
    return 'низкий'

Применим функцию income к столбцу 'total_income' и создадим новый столбец 'income_categories' в таблице data_mod.

In [74]:
data_mod['income_categories'] = data_mod['income_type'].apply(income_type)

Применим к столбцу 'income_categories' метод value_counts(), посмотрим что у нас получилось.

In [75]:
data_mod['income_categories'].value_counts()

средний    11119
высокий     6546
низкий      3860
Name: income_categories, dtype: int64



Создадим таблицу income_debt. Для этого группируем значения столбца 'income_categories' с помощью метода groupby(), далее с помощью метода agg() добавляем еще два столбца: в первом количество значений данной категории, во втором сумма невыплаченных кредитов в данной категории.

In [76]:
income_debt = data_mod.groupby('income_categories').agg({'income_categories':['count'],'debt': ['sum']})

Добавим в таблицу income_debt новый столбец 'ratio', как отношение числа невыплаченных кредитов к числу значений в данной категории.

In [77]:
income_debt['ratio'] = income_debt['debt']['sum'] / income_debt['income_categories']['count']

Выведем таблицу income_debt на экран, отсортируя ее методом sort_values с аргументом 'ratio'. Также с помощью аргумента ascending=False сделаем сортировку по убыванию.

In [78]:
income_debt.sort_values(by='ratio', ascending=False)

,income_categories,debt,ratio
,count,sum,
income_categories,,,
средний,11119,1061,0.095422
высокий,6546,462,0.070577
низкий,3860,218,0.056477


Мы получили таблицу.

Теперь создадим таблицу purpose_debt. Для этого группируем значения столбца 'purpose_lemm' с помощью метода groupby(), далее с помощью метода agg() добавляем еще два столбца: в первом количество значений данной категории, во втором сумма невыплаченных кредитов в данной категории.

In [79]:
purpose_debt = data_mod.groupby('purpose_lemm').agg({'purpose_lemm':['count'],'debt': ['sum']})

Добавим в таблицу income_debt новый столбец 'ratio', как отношение числа невыплаченных кредитов к числу значений в данной категории.

In [80]:
purpose_debt['ratio'] = purpose_debt['debt']['sum'] / purpose_debt['purpose_lemm']['count']

Выведем таблицу purpose_debt на экран, отсортируя ее методом sort_values с аргументом 'ratio'. Также с помощью аргумента ascending=False сделаем сортировку по убыванию.

In [81]:
purpose_debt.sort_values(by='ratio', ascending=False)

,purpose_lemm,debt,ratio
,count,sum,
purpose_lemm,,,
автомобиль,4315,403,0.093395
образование,4022,370,0.091994
свадьба,2348,186,0.079216
недвижимость,6367,474,0.074446
жилье,4473,308,0.068858


Теперь перейдем к построению сводных таблиц.

Для начала создадим в нашей таблице data_mod столбец со значеними "1" для того, чтобы удобно было считать общее количество клиентов определенной категории.

In [82]:
data_mod['count_clients'] = 1

In [83]:
data_mod['count_clients'].value_counts()

1    21525
Name: count_clients, dtype: int64

Приступим к созданию сводной таблицы. В качестве столбцов для группировки выбираем последоватально 'children_categories', 'income_categories', 'family_status', в значениях сводной таблицы выбираем столбцы 'count_clients', 'debt' и применяем функцию sum к значениям сводной таблицы.

In [84]:
data_pivot = data_mod.pivot_table(index=['children_categories', 'income_categories', 'family_status'], values=['count_clients', 'debt'], aggfunc={'count_clients':'sum','debt':'sum'})

Добавим в таблицу data_pivot новый столбец 'ratio', как отношение числа невыплаченных кредитов к числу значений в данной категории. Умножим это значение на 100, чтобы в таблице показать это значение в процентах.

In [85]:
data_pivot['ratio'] = data_pivot['debt'] / data_pivot['count_clients'] * 100

Выведем таблицу data_pivot на экран, применив к ней сортировку методом value_counts() по столбцу ratio. Также применим форматирование вывода значений столбца 'ratio', чтобы значения были в процентах и с 2 знаками после запятой.

In [86]:
data_pivot.sort_values(by='ratio', ascending=False).style.format({'ratio':'{:.2f}%'})

Приступим к созданию второй сводной таблицы. В качестве столбцов для группировки выбираем последоватально 'children_categories', 'income_categories', 'purpose_lemm', в значениях сводной таблицы выбираем столбцы 'count_clients', 'debt' и применяем функцию sum к значениям сводной таблицы.

In [87]:
data_pivot_two = data_mod.pivot_table(index=['children_categories','income_categories', 'purpose_lemm'], values=['count_clients', 'debt'], aggfunc={'count_clients':'sum','debt':'sum'})

Добавим в таблицу data_pivot новый столбец 'ratio', как отношение числа невыплаченных кредитов к числу значений в данной категории. Умножим это значение на 100, чтобы в таблице показать это значение в процентах.

In [88]:
data_pivot_two['ratio'] = data_pivot_two['debt'] / data_pivot_two['count_clients'] * 100

Выведем таблицу data_pivot_two на экран, применив к ней сортировку методом value_counts() по столбцу ratio. Также применим форматирование вывода значений столбца 'ratio', чтобы значения были в процентах и с 2 знаками после запятой.

In [89]:
data_pivot_two.sort_values(by='ratio', ascending=False).style.format({'ratio':'{:.2f}%'})

# Выводы

## Описание проекта

Для выполнения самостоятельного проекта мне были предоставлены данные со статистикой о платёжеспособности клиентов. В ходе проекта нужно было разобраться влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

## Ход работы проекта

В ходе работы над проектом была проанализирована информация о таблице с данными. В частности в данных были обнаружены пропуски, отрицательные значения, которые должны быть положительными, строки с "нулевыми" значениями, различия в регистре значений.

Таблица с данными была подвергнута методам предобработки данных и подготовлена для проведения исследования. Также из данных были удалены строки дубликаты.

Следующим шагом была лемматизация значений столбца *цель кредита* и сведение значений целей кредита к нескольким категориям. Далее была проведена категоризация столбцов *дети*, *семейный статус* и *уровень дохода*.

Затем были составлены таблицы, необходимые для ответов на вопросы задания 2. В конце работы составлены две сводные таблицы, позволяющие сделать дополнительные выводы по проекту.

## Результаты проекта

-  **Зависимость между наличием детей и возвратом кредита в срок.** Были получены следующие результаты: 

Категория клиента | Процент риска
------------------|--------------
        Есть дети | 9,2 %
        Нет детей | 7,5 %

Можно сделать вывод, что наличие детей увеличивает риск невозврата кредита.


-  **Зависимость между семейным положением и возвратом кредита в срок.** Были получены следующие результаты:

   Категория клиента | Процент риска
----------------------|--------------
Не женат / не замужем | 9,7 %
гражданский брак      | 9,3 %
женат / замужем       | 7,5 %         
в разводе             | 7,1 %
вдовец / вдова        | 6,5 %

У клиентов, которые относятся к категории не *женат / не замужем* и *гражданский брак* риск не возвратить кредит выше.

- **Зависимость между уровнем дохода и возвратом кредита в срок.** Были получены следующие результаты.

   Категория клиента | Процент риска
----------------------|--------------
средний уровень дохода | 9,5 %
высокий уровень дохода | 9,3 %
низкий уровень дохода  | 5,6 %        

В данном случае четкой зависимости между уровнем дохода и процентом риска невозврата кредита нет. Возможно, это связано с тем, что не совсем корректно были выбраны категории дохода (по типу работы клиента, а не по реальной заработной плате), возможно, в данной выборке большее влияние на возврат кредита оказывают другие параметры клиентов, нежели уровень дохода.

- **Зависимость между целями кредита и его возвратом в срок.** Были получены следующие результаты.

   Цель кредита | Процент риска
----------------------|--------------
автомобиль | 9,3 %
образование      | 9,2 %
свадьба       | 8,0 %         
недвижимость   | 7,4 %
жилье  | 6,9 %

Кредиты, цели которых связаны с автомобилями и образованием представляют больший риск невозврата по сравнению с остальными целями.

## Общие выводы

В ходе категоризации целей кредита были получена следующая таблица с данными.

   Категория кредита | Процент риска
----------------------|--------------
недвижимость   | 6351
жилье  | 4460
автомобиль | 4306
образование      | 4013
свадьба       | 2324

Можно сделать вывод, что клиенты больше берут кредиты на недвижимость, а на жилье, автомобиль и образование в среднем одинаково, и меньше всего на свадьбу =)

В ходе работы над проектом были составлены две сводные таблицы. Первая таблица состоит из сгруппированных данных *наличие детей*, *уровень дохода* и *семейное положение*, числа клиентов в каждой категории, суммы невыплаченных кредитов в каждой категории и значения невыплаченых кредитов в процентах. 

Из этой таблицы можно сделать вывод, что клиенты с детьми с низким или средним уровнем дохода, находящиеся в гражданском браке, либо не женаты / не замужем, либо вдовцы / вдовы, представляют высокий риск невозврата кредита от **12,6 %** до **14,8 %**.

Вторая таблица состоит из сгруппированных данных *наличие детей*, *уровень дохода* и *цель кредита*, числа клиентов в каждой категории, суммы невыплаченных кредитов в каждой категории и значения невыплаченых кредитов в процентах.

Из второй сводной таблицы можно сделать вывод, что у клиентов с детьми, со средним уровнем дохода, целями кредита которых являются автомобиль, образование или недвижимость, риск невозврата кредита выше и составляет от **10,6 %** до **12,4 %**.

------